In [ ]:
%matplotlib notebook

In [ ]:
import numpy as np
import pandas as pd

<hr style="border-width:3px; border-color:coral"/>
# Timing results for pressure Poisson solver
<hr style="border-width:3px; border-color:coral"/>


Two solver types are reported. In the full matrix system, the discretized system $A\mathbf x = \mathbf b$ is solved. In the Schur complement system, a linear system for the interface values is solved, and a fast FFT-based solver is used for patch solves.  In either the Full or the Schur approaches, GMRES is used as the basic solver (from HYPRE?) along with an an AMG pre-conditioner from HYPRE (BoomerAMG).

Both simulations reported below are then
* Full matrix with HYPRE used as the pre-conditioner for the full system. [Full matrix (HYPRE)](#fullhypre)
* Schur matrix with HYPRE used as the pre-conditioner for the Schur complement system. [Schur matrix (HYPRE)](#schurhypre)

We also include comparisions between HYPRE and the Schur matrix approach [here](#comparisons).
<hr style="border-width:3px; border-color:coral"/>

## Read in spreadsheet data

This is set up to read the "div 2" data.  To read data from other runs, change the four statements below from   "(2)" to "(3)" or "(4)".   

**Note:** Data may not yet be available in the spreadsheet for these other cases. 

In [ ]:
%reload_ext autoreload
%autoreload 2

from results_output import *

In [ ]:
timing_results = pd.read_excel("Timing_results.xlsx",sheet_name=None)
div_factor = 3
df_full_hypre  = timing_results["full_hypre ({:d})".format(div_factor)]
df_schur_hypre = timing_results["schur_hypre ({:d})".format(div_factor)]


# For HYPRE results
cols_setup_hypre = ['matrix_setup','hypre_setup']
df_full_hypre['complete_setup'] = df_full_hypre[cols_setup_hypre].sum(axis=1)
df_schur_hypre['complete_setup'] = df_schur_hypre[cols_setup_hypre].sum(axis=1)


cols_solve_full =['linsolve']
cols_solve_schur = ['linsolve','patch_solve']

<a id='fullhypre'></a>
<hr style="border-width:3px; border-color:coral"/>
# Full matrix (HYPRE) 
<hr style="border-width:3px; border-color:coral"/>

In [ ]:
import seaborn as sns
cm = sns.light_palette("blue", as_cmap=True)

styler = print_table(df_full_hypre,display_table=False);

# Apply gradient coloring;  use low=, high= to extend domain to [xmin-low*(xmax-xmin),xmax+high*(xmax-xmin)]; defaults=0
styler.background_gradient(subset=['complete_solve','complete_setup'],cmap=cm,low=0,high=0.7)

## Full matrix (HYPRE) : Setup

In [ ]:
barh_plot(df_full_hypre,cols_setup_hypre)
plt.title('Full matrix (setup)',fontsize=16);
set_xticks(20,5)

## Full matrix (HYPRE) : Solve

In [ ]:
bar_plot(df_full_hypre,cols_solve_full)
plt.title('Full solve (HYPRE)',fontsize=16);
set_xticks(5,2.5)
plt.xlim([0,20])
plt.xlabel('Time (seconds)');

## Full matrix (HYPRE) : Strong scaling

In [ ]:
strong_scaling(df_full_hypre,field='complete_solve')
plt.title('Full matrix (HYPRE) : Solve');

## Full matrix (HYPRE) : Efficiency

In [ ]:
efficiency(df_full_hypre,'complete_solve')

<a id='schurhypre'></a>
<hr style="border-width:3px; border-color:coral"/>
# Schur matrix (HYPRE) 
<hr style="border-width:3px; border-color:coral"/>

In [ ]:
styler = print_table(df_schur_hypre,display_table=False);

# Apply gradient coloring;  use low=, high= to extend domain to [xmin-low*(xmax-xmin),xmax+high*(xmax-xmin)]; defaults=0
styler.background_gradient(subset=['complete_solve','complete_setup'],cmap=cm,low=0,high=0.5)

## Schur matrix (HYPRE) : Setup

Use HYPRE to solve the Schur complement system

In [ ]:
barh_plot(df_schur_hypre,cols_setup_hypre)
plt.title('Schur matrix (HYPRE) : Setup',fontsize=16);
set_xticks(2,1,fmt='{:.1f}'.format)
plt.xlim([0,7.5]);

## Schur matrix (HYPRE) : Strong scaling (setup)

In [ ]:
strong_scaling(df_schur_hypre,field='complete_setup')
plt.title('Schur matrix (HYPRE) : Strong scaling (setup)');

## Schur matrix (HYPRE) : Setup

In [ ]:
efficiency(df_schur_hypre,'complete_setup')
plt.title('Schur matrix (HYPRE) : Efficiency (%)');

## Schur matrix (HYPRE) : Solve

In [ ]:
barh_plot(df_schur_hypre,cols_solve_schur)

plt.title('Schur matrix (HYPRE) : solve',fontsize=16);
set_xticks(2,1,fmt='{:.1f}'.format)

plt.xlim([0,7.5]);

## Schur matrix (HYPRE) : Strong scaling (solve)

In [ ]:
strong_scaling(df_schur_hypre,field='complete_solve')
plt.title('Schur matrix (HYPRE) : Strong scaling (solve)');

## Schur matrix (HYPRE) : Setup vs solve time 

In [ ]:
bar_plot(df_schur_hypre,['complete_solve','complete_setup'])

plt.xlim([0,7.5])
plt.title('Schur matrix (HYPRE) : Setup vs. solve',fontsize=16);
set_xticks(2,1)
plt.grid(b=True,which='major')

## Schur matrix (HYPRE) : Efficiency

In [ ]:
efficiency(df_schur_hypre,'complete_solve')
plt.title('Schur matrix (HYPRE) : Efficiency (solve)');

<a id='comparisons'></a>
<hr style="border-width:3px; border-color:coral"/>
# Comparisons between full and Schur systems
<hr style="border-width:3px; border-color:coral"/>

In [ ]:
panel = pd.Panel({'Full (HYPRE)' : df_full_hypre, 'Schur (HYPRE)' : df_schur_hypre})
procs = df_full_hypre['p']

## Comparisons : Setup

In [ ]:
df_setup = panel.minor_xs('complete_setup')
df_setup.insert(0,'p',procs)
df_setup

In [ ]:
df_setup.plot(x='p',logx=True,logy=True,style='.-',markersize=10)
plt.xlabel('p')
plt.ylabel('Time (seconds)')
plt.title('Setup time',fontsize=16);
fix_xticks(df_setup['p'].values)

In [ ]:
cols = ['Schur (HYPRE)', 'Full (HYPRE)']
bar_plot(df_setup,cols=cols)
plt.title("Timing (setup)", fontsize=16);
set_xticks(2,1)

In [ ]:
cols = ['Schur (HYPRE)', 'Full (HYPRE)']
bar_plot(df_setup,cols=cols)
plt.title("Timing (setup)", fontsize=16);

## Comparisons : Solve

In [ ]:
df_solve = panel.minor_xs('complete_solve')
df_solve.insert(0,'p',procs)
df_solve

In [ ]:
cols = ['p','Full (HYPRE)','Schur (HYPRE)']
df_solve[cols].plot(x='p',logx=True,logy=False,style='.-',markersize=10)
plt.xlabel('p')
plt.ylabel('Time (seconds)')
plt.title('Solve time (HYPRE)',fontsize=16);
fix_xticks(df_solve['p'].values)

In [ ]:
cols=['Full (HYPRE)','Schur (HYPRE)']
bar_plot(df_solve,cols=cols)
plt.title("Timing (solve)", fontsize=16);
set_xticks(5,2.5)